In [155]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import chromedriver_binary
import csv
import re

In [145]:
driver = webdriver.Chrome()
rent_category = []
num = 1
length = len(rent_category)

while length <= 10:
    print("numの現在の数：ページ数",num)
    print("現在の取得カテゴリ番号",rent_category)
    print("rent_category:length",length)
    driver.get('https://www.f-takken.com/freins/rent/houses/area/items?listtype=houses&lang=ja&location=area&locate%5B0%5D=40131&locate%5B1%5D=40132&locate%5B2%5D=40133&locate%5B3%5D=40134&locate%5B4%5D=40135&locate%5B5%5D=40136&locate%5B6%5D=40137&order1=nn&order2=&limit=20&data_21=0&data_22=0&data_30=0&data_31=0&data_486=&data_486_code=&data_487=&data_487_code=&page={}&currentTabIndex=0'.format(num))
    time.sleep(5)
    # 検索先のページのHTMLを取得
    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')

    # 表示されているカテゴリの番号を全て記憶 
    for element in soup.find_all("li",class_ = "item list-tpl"):
        data_id = element.get("data-id")
        rent_category.append(data_id)
        length = len(rent_category)
        time.sleep(5)
        rent_category = list(set(rent_category))    
    num = num+1
print(rent_category)



numの現在の数：ページ数 1
現在の取得カテゴリ番号 []
rent_category:length 0
['98520113', '98511964', '98534363', '99266113', '99167417', '98415533', '98990005', '99307398', '99308774', '98568710', '99293053', '98333107', '99287205', '99317946', '98389286', '98886218', '99316161', '98514452', '98554977', '99133089']


In [146]:
# 1000個のカテゴリ番号を保存できるまで、次のページに遷移する TODO ○
# driverのpage = {}を変更していく TODO ○
# whileでrent_categoryが1000より少ない時までnumを増やしてpageにformatする ○
#取得の繰り返し ○

#全て取得したら、driverに詳細のページのURLでカテゴリ番号を追記して表示 ○
#表示したら、必要な情報を取得、保存

In [160]:

count = 0
data_set = int(rent_category[count])
print(data_set)
driver.get('https://www.f-takken.com/freins/items/{}'.format(data_set))
time.sleep(5)
# 検索先のページのHTMLを取得
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, 'html.parser')
count +=1

a = soup.find("span",class_ = "price-area")

price = re.sub(r"[^\d.]", "", a.text)

category = soup.find_all("td",class_ = "category")

# 書き込み
with open('RENT.csv', 'w') as csv_file:
    fieldnames = ['家賃','物件番号','情報更新日','次回更新予定日','所在地','建物名','沿線・駅・交通','専有面積','専用庭面積','間取り','間取り内訳','バルコニー','サービスルーム数']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow({'家賃': price})
print(a.text)


98520113


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=62775): Max retries exceeded with url: /session/ad8004ad60b7b0dbbf65b2487867e8b3/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd921182070>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
# ページ遷移
driver.get("https://www.f-takken.com/freins/items/{}".format(search_box))